# Twitter Tocicity using BERT Transformer

URL : https://www.kaggle.com/c/jigsaw-toxic-comment-classification-challenge/overview

In [1]:
from google.colab import drive
drive.mount('/content/drive')

!cp drive/MyDrive/jigsaw-toxic-comment-classification-challenge.zip .
!unzip jigsaw-toxic-comment-classification-challenge.zip
!unzip train.csv.zip
!unzip test.csv.zip

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Archive:  jigsaw-toxic-comment-classification-challenge.zip
replace sample_submission.csv.zip? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: sample_submission.csv.zip  
replace test.csv.zip? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: test.csv.zip            
replace test_labels.csv.zip? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: test_labels.csv.zip     
replace train.csv.zip? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: train.csv.zip           
Archive:  train.csv.zip
replace train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: train.csv               
Archive:  test.csv.zip
replace test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: test.csv                


# Imports and Loading Data

In [9]:
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

In [4]:
data = pd.read_csv("train.csv")
data.head(10)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0
5,00025465d4725e87,"""\n\nCongratulations from me as well, use the ...",0,0,0,0,0,0
6,0002bcb3da6cb337,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,1,1,0,1,0
7,00031b1e95af7921,Your vandalism to the Matt Shirvington article...,0,0,0,0,0,0
8,00037261f536c51d,Sorry if the word 'nonsense' was offensive to ...,0,0,0,0,0,0
9,00040093b2687caa,alignment on this subject and which are contra...,0,0,0,0,0,0


The data is multiclass binary, and one row can have multiple true at the same time.

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 8 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   id             159571 non-null  object
 1   comment_text   159571 non-null  object
 2   toxic          159571 non-null  int64 
 3   severe_toxic   159571 non-null  int64 
 4   obscene        159571 non-null  int64 
 5   threat         159571 non-null  int64 
 6   insult         159571 non-null  int64 
 7   identity_hate  159571 non-null  int64 
dtypes: int64(6), object(2)
memory usage: 9.7+ MB


In [6]:
d = data.describe().T
d['Yes'] = d['mean']*d['count']
d['No'] = d['count'] - d['Yes']
d[['count', 'Yes', 'No']]

,count,Yes,No
toxic,159571.0,15294.0,144277.0
severe_toxic,159571.0,1595.0,157976.0
obscene,159571.0,8449.0,151122.0
threat,159571.0,478.0,159093.0
insult,159571.0,7877.0,151694.0
identity_hate,159571.0,1405.0,158166.0


# Proprocessing

In [7]:
data = data.drop(['id'], axis = 1)
data.head()

,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


# BERT

In [10]:
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string)

preprocessor = hub.KerasLayer(
    "https://tfhub.dev/tensorflow/bert_en_cased_preprocess/3")
encoder_inputs = preprocessor(text_input)

encoder = hub.KerasLayer(
    "https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-256_A-4/2",
    trainable=True)
outputs = encoder(encoder_inputs)

pooled_output = outputs["pooled_output"]
embedding_model = tf.keras.Model(text_input, pooled_output)

## Getting Embeddings from BERT

### Takes a lot of time here
GPU for 5000 observations = 11mins

In [18]:
sz = 5000
X = data['comment_text'][:sz]
Y = data['toxic'][:sz]

embeddings = embedding_model(X)

# Use embeddings in custom model

### Good old Logistic regression

In [20]:
from sklearn.linear_model import LogisticRegression
mdl = LogisticRegression(max_iter = 1000)

mdl.fit(embeddings,Y)
mdl.score(embeddings,Y)

0.9082

### New Dense NN

In [21]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(64, input_shape=(256, )),
    tf.keras.layers.Dense(32, activation = 'relu'),
    tf.keras.layers.Dense(8, activation = 'relu'),
    tf.keras.layers.Dense(1, activation = 'sigmoid')
])

model.compile(
    optimizer = 'adam',
    loss = tf.keras.losses.BinaryCrossentropy(),
    metrics=['accuracy']
)

history  = model.fit(
    embeddings, Y, epochs=100, validation_split = 0.1, shuffle = True 
)

Epoch 1/100
141/141 [==============================] - 1s 4ms/step - loss: 0.3213 - accuracy: 0.8991 - val_loss: 0.3018 - val_accuracy: 0.9000
Epoch 2/100
141/141 [==============================] - 0s 3ms/step - loss: 0.3052 - accuracy: 0.8993 - val_loss: 0.3017 - val_accuracy: 0.9000
Epoch 3/100
141/141 [==============================] - 0s 3ms/step - loss: 0.2995 - accuracy: 0.8998 - val_loss: 0.3012 - val_accuracy: 0.8960
Epoch 4/100
141/141 [==============================] - 0s 3ms/step - loss: 0.2917 - accuracy: 0.9002 - val_loss: 0.3040 - val_accuracy: 0.9040
Epoch 5/100
141/141 [==============================] - 0s 3ms/step - loss: 0.2950 - accuracy: 0.9009 - val_loss: 0.2907 - val_accuracy: 0.8940
Epoch 6/100
141/141 [==============================] - 0s 3ms/step - loss: 0.2855 - accuracy: 0.9016 - val_loss: 0.2976 - val_accuracy: 0.8940
Epoch 7/100
141/141 [==============================] - 0s 3ms/step - loss: 0.2827 - accuracy: 0.9020 - val_loss: 0.3060 - val_accuracy: 0.8840